In [1]:
# main.ipynb
# Archivo base en el que se maneja la experimentación del diseño metodológico.
# Objetivo general: Utilizar los datos publicos (ubicados en la carpeta /data/)
#     de municipios en los que existe información para diseñar una estrategia
#     para distribuir alimentos en el territorio nacional.
# Objetivos específicos:

#     ---pronostico_poblacional---
#     [IA]1. Determinar por municipio que herramienta es mejor para pronosticar
#            la población en 10 años a partir del año actual.
#               a. Regresión Lineal Multiple || Multiple Linear Regression.
#               b. Arboles de Regresión || Regression Tree.
#               c. Máquinas de Vectores de Soporte || Support Vector Machine.
#               d. Bosques Aleatorios || Random Forest Regression.
#               e. Redes neurales || Deep Learning.
#               f. Regresión Tradicional.
#     ---capacidad_y_costo---
#     [IO]2. Determinar las capacidades y costos de almacenamiento de alimentos
#            en cada municipio utilizando los datos del pronóstico, los datos
#            abiertos y consideraciones de los analístas.
#     ---pronosticar_capacidad_y_costo---
#     [IA]3. Entrenar un clasificador desde los datos de capacidades y costos
#            de almacenamiento por municipio.
#               a. Arboles de Decision || Decision Tree.
#               b. Análisis Discriminante Lineal || Linear Discriminant Analysis.
#               c. Regresión Logística || Logistic Regression.
#               d. Máquinas de Vectores de Soporte || Support Vector Machine.
#               e. Redes Neurales || Deep Learning.
#               f. Análisis de frecuencias. [Aún está en desarrollo la idea]
#     ---cantidad_de_clusteres---
#     [IO]4. Proponer una cantidad de clusteres que permitan disminuir el costo
#            computacional del algoritmo optimizador y compararlo con los métodos
#            tradicionales.
#     ---generar_clusteres---
#     [IA]5. Dividir los municipios en clústeres, rectificando que es viable
#            satisfacer la demanda de alimentos con la capacidad instalada del
#            municipio.
#               a. k-means.
#               b. Mapa Autoorganizado || Self-Organizing Map.
#               c. Agrupamiento Jerárquico || Agglomerative Clustering.
#               d. DBSCAN.
#     ---solucionar_cflp---
#     [IO]6. Resolver el cflp para diferentes escenarios.
#               a. Solución ingenua (todos los municipios tienen un centro de
#                  distribución).
#               b. Datos completos sin clusterizar.
#               c. Dividido por clústers.
#


In [2]:
# Manejo básico de archivos
import os, sys, warnings

# Registro del tiempo
import time

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import MatplotlibDeprecationWarning
from sklearn.preprocessing import MinMaxScaler

# Manejo de datos
import pandas as pd
import numpy as np
import pulp as pl

# Funciones personalizadas de /funciones/funciones.py, /funciones/alistamiento.py, /funciones/miscelaneas.py
from funciones.funciones import *
from funciones.alistamiento import *
from funciones.miscelaneas import *

# funciones del flujo de trabajo
from funciones.pronostico_poblacional import *
from funciones.capacidad_y_costo import *
from funciones.predecir_capacidad_y_costo import *
from funciones.cantidad_de_clusteres import *
from funciones.generacion_de_clusteres import *


In [3]:
# Funciones


def cargar_datos_para_mapa(municipios, solucion):
    print(f"Creando mapa para {solucion}")
    # cargar los datos
    print(f"    Cargando datos", end="\r")
    X = pd.read_excel(
        f"resultados/tablas/solucionar_cflp/soluciones/{solucion}.xlsx",
        sheet_name="X",
        index_col=0,
    )
    Y = pd.read_excel(
        f"resultados/tablas/solucionar_cflp/soluciones/{solucion}.xlsx",
        sheet_name="Y",
        index_col=0,
    )
    Y["lat"] = municipios.loc[Y.index, "lat"]
    Y["lon"] = municipios.loc[Y.index, "lon"]
    # Normalizar los datos de X por columna
    print(f"    Normalizando datos", end="\r")
    for col in X.columns:
        X[col] = MinMaxScaler().fit_transform(X[[col]])
    print(f"    Datos cargados")
    return Y, X


def hacer_mapas():
    """
    Función que grafica los mapas de los resultados obtenidos.
    para los archivos en:
    resultados/tablas/solucionar_cflp/soluciones/datos_completos-agglomerative.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_completos-dbscan.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_completos-ingenua.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_completos-kmeans.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_completos-sin-clusterizar.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_completos-som.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_imperfectos-agglomerative.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_imperfectos-dbscan.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_imperfectos-ingenua.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_imperfectos-kmeans.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_imperfectos-sin-clusterizar.xlsx
    resultados/tablas/solucionar_cflp/soluciones/datos_imperfectos-som.xlsx

    Cada uno es un archivo de excel con las siguientes hojas:
    1. 'Y': columnas = ['divipola', 'Y', 'cluster', 'municipio']
    2. 'X': columnas = codigos divipola de los municipios, indices = codigos divipola de los municipios

    Las latitudes y longitudes de los municipios están en el archivo:
    data/municipios, la columna de divipola es 'dpmp'

    para crear el mapa, se usa Basemap de matplotlib.

    cada mapa se guarda en resultados/mapas/{tipo_de_dato}-{tipo_de_solucion}.png

    Los mapas tienen puntos de color diferente dependiendo del cluster al que pertenecen.
    los mapas tienen un Triangulo invertido en las ubicaciones con 'Y' = 1.
    Los mapas tienen una Linea que va desde el centro de distribución al municipio. que es la variable 'X', se normaliza de 0-1 por columnas.
    las lineas tienen un grosor proporcional a la cantidad de alimentos que se envían.
    Las lineas tienen un color diferente dependiendo del cluster al que pertenecen.
    Los labels de los mapas son circulo: Municipio, Triangulo: CD, Linea: Ruta.

    Color del agua: #A0D9D3
    Color de la tierra: #51A66F
    """
    # MatplotlibDeprecationWarning
    warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
    lista_de_soluciones = [
        # "datos_completos-agglomerative",
        # "datos_completos-dbscan",
        # "datos_completos-ingenua",
        # "datos_completos-kmeans",
        # "datos_completos-sin-clusterizar",
        # "datos_completos-som",
        "datos_imperfectos-agglomerative",
        "datos_imperfectos-dbscan",
        # "datos_imperfectos-ingenua",
        "datos_imperfectos-kmeans",
        # "datos_imperfectos-sin-clusterizar",
        "datos_imperfectos-som",
    ]
    municipios = pd.read_csv("data/municipios.csv", index_col="dpmp")
    # Lista de 30 colores que no incluyen verde
    # colores =
    for solucion in lista_de_soluciones:
        Y, X = cargar_datos_para_mapa(municipios, solucion)
        # crear el mapa
        print(f"    Creando mapa", end="\r")
        fig = plt.figure(figsize=(20, 20))
        m = Basemap(
            projection="merc",
            llcrnrlat=-5,
            urcrnrlat=15,
            llcrnrlon=-80,
            urcrnrlon=-65,
            resolution="i",
        )
        m.drawcountries()
        m.drawcoastlines()
        m.drawmapboundary(fill_color="#A0D9D3")
        m.fillcontinents(color="#d4fae2", lake_color="#A0D9D3")
        m.drawparallels(np.arange(-90, 90, 5), labels=[1, 0, 0, 0])
        m.drawmeridians(np.arange(-180, 180, 5), labels=[0, 0, 0, 1])
        m.drawmapscale(-79, -4, -79, -4, 500, barstyle="fancy")
        m.drawmapboundary(fill_color="#A0D9D3")
        # Dibujar los puntos de los municipios
        for cluster in Y["cluster"].unique():
            # Definir los colores de los clusters que actualmente son numeros a colores de la escalas de colores de matplotlib
            colores = plt.cm.get_cmap("tab20", len(Y["cluster"].unique()))
            colores = colores.colors
            colores = [
                f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}"
                for r, g, b, _ in colores
            ]
            if solucion in [
                "datos_completos-ingenua",
                "datos_imperfectos-ingenua",
                "datos_completos-sin-clusterizar",
                "datos_imperfectos-sin-clusterizar",
            ]:
                color_puntos = "#1e3be3"
                color_triangulos = "#1e3be3".replace("#", "#33")
            else:
                color_puntos = colores[cluster]
                color_triangulos = colores[cluster].replace("#", "#33")
            for origen in X.index:
                for destino in X.columns:
                    if float(X.loc[origen, destino]) > 0:
                        m.plot(
                            [Y.loc[origen, "lon"], Y.loc[destino, "lon"]],
                            [Y.loc[origen, "lat"], Y.loc[destino, "lat"]],
                            latlon=True,
                            c="#F28D52",
                            linewidth=float(X.loc[origen, destino]) * 5,
                            label=f"Ruta",
                        )
            m.scatter(
                Y.loc[Y["cluster"] == cluster, "lon"],
                Y.loc[Y["cluster"] == cluster, "lat"],
                latlon=True,
                s=100,
                edgecolors="k",
                c=color_puntos,
                label=f"Municipio",
                zorder=10,
            )
            # Dibujar los triangulos de los centros de distribución
            m.scatter(
                Y["lon"].loc[Y["Y"] == 1].loc[Y["cluster"] == cluster],
                Y["lat"].loc[Y["Y"] == 1].loc[Y["cluster"] == cluster],
                latlon=True,
                s=100,
                edgecolors="k",
                # mismo color pero más oscuro
                c=color_triangulos,
                marker="v",
                label=f"CD",
                zorder=15,
            )
            # Dibujar las lineas de las rutas
        # Dibujar uno de cada uno fuera del mapa para darle el estilo de las etiquetas
        plt.plot(
            [0, 1],
            [0, 1],
            c="#8C8C8C",
            linewidth=5,
            label=f"Ruta",
        )
        plt.scatter(
            [0],
            [0],
            s=100,
            edgecolors="k",
            c="#8C8C8C",
            label=f"Municipio",
            zorder=10,
        )
        plt.scatter(
            [0],
            [0],
            s=100,
            edgecolors="k",
            c="#8C8C8C".replace("#", "#33"),
            marker="v",
            label=f"CD",
            zorder=10,
        )
        # Hacer la leyenda sin repetir y hacerlas con color blanco de relleno y edgecolor negro
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())
        plt.savefig(f"resultados/mapas/{solucion}.png")
        # plt.show()
        plt.close()


In [4]:
# PARAMETROS

comida_per_capita = 0.00087617  # 0.87617 kg por persona / día
densidad_de_alimentos = 537 / 1000  # 537 kg por m3 (5 metros de altura)
RANDOM_SEED = 11

crear_estructura_de_archivos()
procesar_datos_completos(comida_per_capita, densidad_de_alimentos)
procesar_datos_imperfectos()

pronostico_poblacional(RANDOM_SEED)
capacidad_y_costo(comida_per_capita)
cantidad_de_clusteres(
    RANDOM_SEED, tiempo_maximo=60 * 10, rango_de_muestras=np.linspace(0.1, 0.3, 12)
)
generar_clusteres(RANDOM_SEED, n_cluster_completo=6, n_cluster_imperfecto=6)
solucionar_cflp(comida_per_capita, tiempo_maximo=60 * 60)
hacer_mapas()


Procesando los DATOS COMPLETOS
Los datos ya fueron procesados
Procesando los DATOS IMPERFECTOS
Los datos ya fueron procesados

El proceso de pronóstico poblacional ya fue realizado

La capacidad y costos ya fueron calculados
Proponiendo k para datos_completos
  Terminado para datos_completos
Proponiendo k para datos_imperfectos
  Terminado para datos_imperfectos
Generando clusteres para datos_completos
    kmeans, progreso: 25.00%
    som, progreso: 50.00%
    agglomerative, progreso: 75.00%
    dbscan, progreso: 100.00%
Generando clusteres para datos_imperfectos
    kmeans, progreso: 25.00%
    som, progreso: 50.00%
    agglomerative, progreso: 75.00%
    dbscan, progreso: 100.00%
Resolviendo el problema para datos_imperfectos
        Procesando modelo kmeanszada
        Se detuvo por tiempo------------------------------------------------------------------------
        Se detuvo por tiempo------------------------------------------------------------------------
        Se detuvo por t